In [713]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [714]:
import os
print(os.getcwd())

C:\Users\guswh\Desktop\data-analysis\dacon_housePrediction\src


In [715]:
train = pd.read_csv('../data/prep/train3.csv')
test = pd.read_csv('../data/prep/test3.csv')
submission = pd.read_csv('../data/raw/housing/sample_submission.csv')

In [716]:
train

,id,Overall.Qual,Gr.Liv.Area,Exter.Qual,Garage.Cars,Garage.Area,Kitchen.Qual,Total.Bsmt.SF,X1st.Flr.SF,Bsmt.Qual,Full.Bath,Year.Built,comb.Qual,Garage.Yr.Blt_cal,Year.Remod.Add_cal,Year.Built_cal,comb.Built,comb.Area,total.Price.Index,target
0,1,2.397895,7.780303,1.791759,1.386294,6.876265,1.791759,7.780303,7.780303,1.791759,1.098612,7.602900,2.772589,2.197225,2.197225,2.197225,2.833213,9.005160,14.015673,12.864243
1,2,2.079442,7.210080,1.609438,1.098612,6.146329,1.609438,7.210080,7.210080,1.791759,1.098612,7.604396,2.639057,1.791759,1.609438,1.791759,2.302585,8.416931,12.927572,12.175619
2,3,1.791759,6.803505,1.386294,0.693147,5.666427,1.386294,6.762730,6.803505,1.386294,0.693147,7.584773,2.302585,3.806662,3.806662,3.806662,4.488636,7.990577,11.796908,11.719948
3,4,1.791759,7.069023,1.386294,1.098612,6.357842,1.609438,6.523562,6.523562,1.386294,0.693147,7.550135,2.397895,2.484907,1.791759,4.718499,4.762174,8.042699,11.954407,11.813037
4,5,2.079442,7.580189,1.609438,1.386294,6.842683,1.609438,6.934397,6.934397,1.609438,1.098612,7.603898,2.564949,1.945910,1.945910,1.945910,2.564949,8.506537,12.937154,12.429220
5,6,2.197225,7.585281,1.609438,1.386294,6.523562,1.791759,6.652863,6.652863,1.791759,1.098612,7.605890,2.708050,1.098612,0.693147,1.098612,1.386294,8.342125,13.060388,12.504327
6,7,1.945910,7.299121,1.386294,1.098612,6.093570,1.386294,7.299121,7.299121,1.386294,0.693147,7.579679,2.302585,4.007333,4.007333,4.007333,4.691348,8.492286,12.481068,11.960818
7,8,2.197225,7.833996,1.609438,1.098612,6.297109,1.609438,7.833996,7.833996,1.609438,1.098612,7.591862,2.564949,3.433987,3.433987,3.433987,4.110874,9.001469,13.565696,12.535380
8,9,2.197225,7.882315,1.609438,1.386294,6.616065,1.609438,7.299797,7.323831,1.791759,1.098612,7.601902,2.639057,2.397895,2.302585,2.397895,2.995732,8.762490,13.406725,12.950984
9,10,2.197225,7.273093,1.609438,1.098612,6.148468,1.609438,7.267525,7.273093,1.791759,1.098612,7.602900,2.639057,2.197225,2.197225,2.197225,2.833213,8.472196,13.116380,12.356650


In [717]:
train = train.iloc[:, 1:]
test = test.iloc[:, 1:]

In [718]:
X = train.drop('target', axis = 1)
#y = np.log1p(train.target)
y = train.target
target = test[X.columns]

In [719]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from catboost import CatBoostRegressor, Pool
from ngboost import NGBRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold

In [720]:
def NMAE(true, pred) -> float:
    mae = np.mean(np.abs(true - pred))
    score = mae / np.mean(np.abs(true))
    return score

In [721]:
nmae_score = make_scorer(NMAE, greater_is_better=False)

In [722]:
kf = KFold(n_splits = 5, random_state = 42, shuffle = True)

In [723]:
# ngboost
ngb_pred = np.zeros(target.shape[0])
ngb_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    print(f'{n + 1} FOLD Training.....')
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    ngb = NGBRegressor(random_state = 42, n_estimators = 1000, verbose = 0, learning_rate = 0.03)
    ngb.fit(tr_x, tr_y, val_x, val_y, early_stopping_rounds = 300)
    
    val_pred = np.expm1(ngb.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    ngb_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}\n')
    
    target_data = Pool(data = target, label = None)
    fold_pred = ngb.predict(target) / 5
    ngb_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(ngb_val)} & std = {np.std(ngb_val)}')

1 FOLD Training.....
1 FOLD NMAE = 0.09539982231227842

2 FOLD Training.....
2 FOLD NMAE = 0.09418575013673591

3 FOLD Training.....
3 FOLD NMAE = 0.09024688922438878

4 FOLD Training.....
4 FOLD NMAE = 0.09565737004996616

5 FOLD Training.....
5 FOLD NMAE = 0.09826534816947649

10FOLD Mean of NMAE = 0.09475103597856915 & std = 0.0026153256545644026


In [724]:
submission['target'] = ngb_pred

In [725]:
submission

,id,target
0,1,12.697127
1,2,11.757677
2,3,12.090712
3,4,12.422429
4,5,11.826303
5,6,11.882686
6,7,12.116545
7,8,11.719650
8,9,11.842736
9,10,12.628120


In [726]:
submission.to_csv("../out/ngb/ngb1.csv", header = True, index = False)